In [80]:
"1234"[:2]

'12'

In [90]:
import exif 

def get_exif(path,selected_keys=None):
    img_exif = None
    exif_info = {}
    with open(img_path, 'rb') as img_file:
        img_exif = exif.Image(img_file)
    if not selected_keys:
        all_exif_keys = img_exif.list_all()
    else:
        all_exif_keys = selected_keys
    for key in all_exif_keys:
        exif_info[key] = img_exif.get(key)
    return exif_info

def get_real_eqvl_focal_len(h,w,exif_info):
    orig_resolution_index = {"SONY: ILCE-1":[(8640,5760)],
                             "LEICA CAMERA AG: LEICA Q2": [(8368,5584)],
                             "RICOH IMAGING COMPANY, LTD.: RICOH GR IIIx":[(6000,4000)],
                             "Apple: iPhone 14 Pro Max":[(8064,6048),(4032,3024)],
                             "Apple: iPhone 13 Pro":[(4032,3024)],
                             "Panasonic: DMC-ZS110":[(5472,3648)]

                            } # w,h
    camera_model = exif_info['make']+": "+exif_info["model"]
    if camera_model not in orig_resolution_index:
        raise ValueError("unknown camera model: [{}]".format(camera_model))
    long_edge_crop_ratio = max(h,w)/max(orig_resolution_index[camera_model][0])
    # iPhone special processing
    if long_edge_crop_ratio<0.5 and camera_model == "Apple: iPhone 14 Pro Max":
        long_edge_crop_ratio = max(h,w)/max(orig_resolution_index[camera_model][1])
        
    # calc real eq focal len
    uncropped_fl = exif_info["focal_length_in_35mm_film"]
    # ff camera use true fl
    if camera_model in ["LEICA CAMERA AG: LEICA Q2","SONY: ILCE-1"]:
        uncropped_fl = exif_info["focal_length"]
    
    real_fl = uncropped_fl/long_edge_crop_ratio

    # sony apsc lens

    return {"camera": camera_model,
            "lens":exif_info['lens_model'],
            "orig_fl":uncropped_fl,
            "real_fl":real_fl,
            "long_edge_crop_ratio":long_edge_crop_ratio
           }

    
    

In [91]:
# TODO
# Enhanced
from glob import glob 
from os.path import join
from pprint import pprint
import cv2
from tqdm import tqdm

folder_path = '/Users/htplex/Pictures/Lightroom Saved Photos/2022-3s-preview/'
useful_exif_keys = ["datetime_original","exposure_bias_value","exposure_mode","exposure_program",
                    "exposure_time","f_number","focal_length","focal_length_in_35mm_film",
                    "lens_model","lens_specification","make","model",
                    "photographic_sensitivity","recommended_exposure_index",
                   ]

img_paths = sorted(glob(join(folder_path,"*.jpg")))
all_better_exif = []
for img_path in tqdm(img_paths):
    # print(img_path)
    img = cv2.imread(img_path)
    h,w = img.shape[:2]
    exif_info = get_exif(img_path,useful_exif_keys)
    # pprint(exif_info)
    
    # init better exif
    # step1: real_eq
    better_exif = get_real_eqvl_focal_len(h,w,exif_info)
    all_better_exif.append(better_exif)

    # break
        
        

100%|█████████████████████████████████████████| 717/717 [02:27<00:00,  4.86it/s]
